# Изучаем обзоры на наушники

Я решил использовать отзывы на полноразмерные и накладные наушники на сайте магазина DNS.

In [1]:
from bs4 import BeautifulSoup
import nltk, re, pymorphy2
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

Выкачать с помощью **requests** не получилось, сайт упорно сопротивлялся, несмотря на прокси и **fake_useragent** (а это был уже далеко не первый сайт, который я пытался выкачать), поэтому я вручную загрузил несколько html-страниц и дальше начал парсить обычным образом.

Вот пример такой страницы: https://www.dns-shop.ru/product/e1e6b9b0b3e63361/provodnaa-garnitura-a4tech-bloody-g500-cernyj/opinion/.

In [2]:
pages = []

i = 1
while True:
    try:
        with open(f"p{i}.html", "r", encoding="utf-8") as f:
            pages.append(f.read())
        i += 1
    except FileNotFoundError:
        break

Вынимаем данные из html-страницы. В системе отзывов DNS есть возможность написать отзыв по формату «Достоинства», «Недостатки» и «Комментарии», и каждый раздел оформляется в html контейнером. Я вынимал их содержимое и складывал вместе без разбору, как будто это обычный текст. Эти подзаголовки, оформленные тоже особым образом, я удалил.

Отзывы в DNS используют систему звёзд, всего максимально звёзд пять, но благодаря особенностям html пять звёзд удобно конвертируются в число 10. То есть рейтинг может иметь значения 2, 4, 6, 8, 10.

In [3]:
def extract_data(page, reviews=[]):

    page_soup = BeautifulSoup(page, "html.parser")
    
    for container in page_soup.find_all("div", {"class": "ow-opinion ow-opinions__item", "data-role": "opinion"}) + page_soup.find_all("div", {"class": "ow-opinion ow-opinion_popular ow-opinions__item", "data-role": "opinion"}):
        
        stars = len(container.find_all("span", {"class": "star-rating__star", "data-state": "selected"}))
        
        text_chunks = container.find_all("div", {"class": "ow-opinion__text-desc"})
        
        texts = []
        for i in range(len(text_chunks)):
            
            stroki = []
            for stroka in text_chunks[i].find_all("p"):
                stroki.append(stroka.text)
            
            text_chunks[i] = "\n".join(stroki)
            if not text_chunks[i] in ("{{Достоинства}}", "{{Недостатки}}", "{{Комментарий}}"):
                texts.append(text_chunks[i])
                
        texts = "\n".join(texts)
        
        if len(texts) > 10:
            reviews.append({"text": texts, "rating": stars})
    return reviews

In [4]:
reviews = []

for page in pages:
    reviews = extract_data(page, reviews)
    
print(len(reviews))
#print(reviews)

215


Токенизируем и лемматизируем все тексты. Для токенизации используется **nltk**, для лемматизации **pymorphy2**.

In [5]:
def tokenize_review(review):
    tokens = nltk.word_tokenize(review["text"])
    review["tokens"] = []
    
    for token in tokens:
        if re.search("[А-Яа-яЁё]", token) is not None:
            review["tokens"].append(token.strip("-"))
    
    return review

def lemmatize_review(review):
    for i in range(len(review["tokens"])):
        review["tokens"][i] = morph.parse(review["tokens"][i])[0].normal_form
    
    filtered_tokens = []
    for i in range(len(review["tokens"])):
        if review["tokens"][i] not in russian_stopwords:
            filtered_tokens.append(review["tokens"][i])
    
    review["tokens"] = filtered_tokens
    
    return review

def preedit(review):
    review = tokenize_review(review)
    review = lemmatize_review(review)
    return review

In [6]:
for i in range(len(reviews)):
    reviews[i] = preedit(reviews[i])
    
print(str(reviews)[:3000])

[{'text': 'Цена = качество\nПока не поняла, если они за такую цену\nВыбирала в этом ценовом сегменте, остановилась на этих из-за положительных отзывов. Купила не для частого прослушивания, например, пока в автобусе едешь или голосовое смс прослушать, вот. Для этого наушники в самый раз. Заодно купила оригинальный чехол, удобно, мне нравится.', 'rating': 10, 'tokens': ['цена', 'качество', 'пока', 'понять', 'цена', 'выбирать', 'это', 'ценовый', 'сегмент', 'остановиться', 'из-за', 'положительный', 'отзыв', 'купить', 'частый', 'прослушивание', 'например', 'пока', 'автобус', 'ехать', 'голосовой', 'смс', 'прослушать', 'это', 'наушник', 'самый', 'заодно', 'купить', 'оригинальный', 'чехол', 'удобно', 'нравиться']}, {'text': 'Коробка была в пленке, однако попытка зарядить наушники завершилась неудачей, кейс не реагирует никак. Пойду сдавать по гарантии.', 'rating': 2, 'tokens': ['коробка', 'плёнка', 'однако', 'попытка', 'зарядить', 'наушник', 'завершиться', 'неудача', 'кейс', 'реагировать', 'ни

Делаем словарь частотных лемм. Переменная *minimum* обозначает минимальное количество повторений леммы в тексте для учёта.

In [7]:
def make_freq_dict(reviews, minimum):
    all_tokens = {}
    for review in reviews:
        for token in review["tokens"]:
            if token in all_tokens:
                all_tokens[token].append(review["rating"])
            else:
                all_tokens[token] = [review["rating"]]
    
    popular_tokens = {}
    for token in all_tokens:
        if len(all_tokens[token]) >= minimum:
            all_tokens[token].sort()
            popular_tokens[token] = all_tokens[token]
    
    return popular_tokens

popular_tokens = make_freq_dict(reviews, minimum=4)
print(str(popular_tokens)[:3000])

{'цена': [2, 2, 2, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], 'качество': [2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], 'пока': [2, 2, 4, 4, 4, 6, 6, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], 'понять': [4, 6, 6, 8, 10, 10], 'выбирать': [8, 8, 10, 10, 10, 10, 10, 10], 'это': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

Пытаемся присвоить каждой лемме значение положительной или отрицательной окраски.

Чтобы максимально выделить именно ярко окрашенные леммы, фильтруем по дисперсии (чтобы лемма не встречалась одинаково часто в отзывах с разной окраской) и медиане (чтобы медиана была явно большая или маленькая).

Для определения «положительного» и «отрицательного» были на глазок подобраны значения: положительные леммы — это такие, которые встречаются в отзывах со средним рейтингом больше или равном 9, а отрицательные встречаются в отзывах со средним меньше или равно 6 (положительных отзывов гораздо больше, поэтому так сдвинуто к положительному полюсу).

In [8]:
def assign_polarity(tokens):
    
    def count_mean(a):
        return round(sum(a)/len(a), 2)
    
    def count_median(a):
        i = int(len(a)//2)
        return a[i] if len(a) % 2 == 1 else count_mean([a[i], a[i-1]])
    
    def count_dispersion(a, mean):
        deviations = []
        for rating in a:
            deviations.append((rating-mean)**2)
        return count_mean(deviations)
    
    markers = {"positive": set(), "negative": set()}
    
    for token in tokens:
        mean = count_mean(tokens[token])
        median = count_median(tokens[token])
        dispersion = count_dispersion(tokens[token], mean)
        
        if dispersion < 3:
            if median >= 9 and len(tokens[token]):  #            < - - - - - - - порог строгой положительности для рейтинга
                markers["positive"].add(token)
            elif median <= 6:
                markers["negative"].add(token)  #                < - - - - - - - порог строгой отрицательности для рейтинга
    
    return markers

markers = assign_polarity(popular_tokens)
print(markers)

{'positive': {'включение', 'противник', 'чистый', 'удобно', 'длительный', 'нравиться', 'шаг', 'ощущаться', 'каждый', 'высота', 'настроить', 'басс', 'крутой', 'отсутствие', 'дискорд', 'среднее', 'диапазон', 'довольный', 'вполне', 'смотреться', 'удобство', 'пожалеть', 'выходить', 'крепкий', 'глянцевый', 'принцип', 'немного', 'особенно', 'реально', 'сборка', 'отличный', 'разговор', 'жаловаться', 'обнаружить', 'единственный', 'выбирать', 'супер', 'правда', 'шея', 'вариант', 'стена', 'ровно', 'подвести', 'хватать', 'топ', 'сын', 'комментарий', 'прекрасный', 'сравнивать', 'прочный', 'передавать', 'надёжный'}, 'negative': {'начаться', 'неудобный', 'писк', 'тугой', 'технология', 'вылезти', 'звукоизоляция', 'крайне', 'амбошюр', 'сдать', 'подушка', 'прийти', 'брак'}}


Получился набор положительных и отрицательных лемм.

Попробуем протестировать на пяти html-файлах, которые я тоже заранее скачал, но которые в обучении не участвовали. Распарсим их таким же образом.

In [9]:
test_pages = []

for x in "ABCDE":
    with open(f"p{x}.html", "r", encoding="utf-8") as f:
        test_pages.append(f.read())
        
test_reviews = []
for test_page in test_pages:
    test_reviews = extract_data(test_page, test_reviews)

for i in range(len(test_reviews)):
    test_reviews[i] = preedit(test_reviews[i])
    
print(str(test_reviews)[:3000])

[{'text': '+ Звук хороший.\n+ Аккуратный дизайн.\n+ Шумодав работает (интересная функция).\n+ Приятная девшука шепчет (Power On, Off и т.д.) при нажатии на кнопку вкл.\n+ Не упадут с ушей, гарантировано.\n- Эти наушники однозначно нацелены на использование для смартфонов, аудио-плееров, авиасамолетов.\n- В комплекте за такую стоимость НЕТ Bluetooth Устройства для PC.\n- Невозможно выключить шумодав вручную без приложения на Android Sony Head....\n- Амбушюры не внушают долгосрочного доверия.\n- Материал обертки ткани и сам ободок, дешевее некуда.\n- Шнур для ПК: это прям максимальная экономия на проводе. 40-60 см т.е. меньше метра. Вам надо сидеть рядом с системником в подмышку его.\n- Шнур для зарядки 7-10 см В КОМПЛЕКТЕ ДАЖЕ ЗАРЯДНИК НЕ ПОЛОЖИЛИ! ЭКОНОМИЯ!\n- Давят зачетно.\n- Не стоят они столько.\nВЫ ТОЛЬКО ПОДУМАЙТЕ!!!! НЕТ УСТРОЙСТВА BLUETOOTH В КОМПЛЕКТЕ!!! ЕГО НЕТ! ПОКУПАТЬ НУЖНО ОТДЕЛЬНО.', 'rating': 6, 'tokens': ['звук', 'хороший', 'аккуратный', 'дизайн', 'шумодать', 'работать

Тут опять непонятно, что вообще значит «положительный» рейтинг, а что «отрицательный», особенно учитывая, что выше мы пытались не учитывать нейтральные леммы, а здесь нам нужно про каждый отзыв сказать, положительный он или негативный, и третьего не дано. Я опять на глазок примерил, что больше 7 (то есть 8 и 10) — это довольно положительно.

Для каждого отзыва я считаю сумму полярностей встретившихся в нём лемм («положительные» токены дают +1, а отрицательные — -1). Вольным образом установил некоторый порог для этой суммы: если сумма полярностей больше или равна 0, то считаем, что отзыв положительный.

In [10]:
def predict_mood(test_review):
    counter = 0
    for token in test_review["tokens"]:
        if token in markers["positive"]:
            counter += 1
        elif token in markers["negative"]:
            counter -= 1
    
    if counter >= 0:  #                     < - - - - - - - порог положительности для суммы полярностей
        return "positive"
    return "negative"

correct = 0
for test_review in test_reviews:
    if test_review["rating"] > 7:  #        < - - - - - - - порог положительности для реального рейтинга
        rating = "positive"
    else:
        rating = "negative"
    
    if rating == predict_mood(test_review):
        correct += 1

accuracy = round(correct/len(test_reviews), 2)
accuracy

0.69

### Что пошло не так?

Ирония в том, что изначально у меня было около десяти файлов для обучения и трёх файлов для тестирования, и *accuracy* выходила больше 60%. Я захотел её улучшить, подумал, что это потому, что я скачивал только отзывы на продукты с большими средними отзывами (не специально, просто DNS мне подсунул такие продукты). Я накачал ещё больше данных, добавив много наушников с низким средним рейтингом; показатель не поднялся, а, наоборот, опустился до 55%. Потом я заменил число порога для *counter* (было >=1, стало >=0), и показатель взлетел до 69%. Не могу понять, так правда логичнее или это читерство. (Это отлично работает, потому что положительных отзывов всё ещё больше, чем отрицательных, и если опустить порог «положительности», то предсказание будет более точным просто потому, что положительных отзывов в выборке больше.) Тем не менее, я верю в силу больших данных, и думаю, что если сделать, к примеру, 100/20 файлов, то будет гораздо лучше в любом случае.

Ещё было бы круто более точно рассчитывать позитивность/негативность в зависимости от распределения рейтингов по выборке, рассчитывать минимальный порог для количества частотных лемм в зависимости от количества уникальных лемм.